In [ ]:
#import spacy.cli
#spacy.cli.download("en_core_web_lg")

In [ ]:
#!gsutil -q -m cp gs://my-bucket-name/en_core_web_lg /content/drive/My\ Drive/

In [ ]:
#import en_core_web_lg
#nlp = en_core_web_lg.load()

In [ ]:
#!python -m spacy download en_core_web_sm

In [ ]:
#nlp = en_core_web_sm.load()

In [ ]:
#!python -m spacy download en_core_web_lg

In [ ]:
#!python -m spacy link en_core_web_lg en || true

In [ ]:
#NER Packages
import plac
import random
import time
import spacy
#import en_core_web_lg
import en_core_web_sm
import os
import traceback
from __future__ import unicode_literals, print_function
from pathlib import Path
from spacy.util import minibatch, compounding
from spacy import displacy
from google.colab import drive
from datetime import datetime 
from google.colab import files 

In [ ]:
#needed objects creation
nlp = spacy.load("en_core_web_sm")

In [ ]:
#Model_attributes
LABEL=list()
model_training_time=0
#model="en_core_web_lg"
model="en_core_web_sm"

#new_model_name
AI_model_name="Artificial_Intelligence_Model"
SW_model_name="Software_Engineering_Model"
PL1_model_name="Programming_Language_1_Model"

#output_dir="/content/gdrive/My Drive/Trained_NER_Models/Artificial_Intelligence_Trained_Model/"
output_dir_AI_version1="/content/gdrive/My Drive/Trained_NER_Models/Artificial_Intelligence_Trained_Model/version1"
output_dir_AI_version2="/content/gdrive/My Drive/Trained_NER_Models/Artificial_Intelligence_Trained_Model/version2"
output_dir_AI_version3="/content/gdrive/My Drive/Trained_NER_Models/Artificial_Intelligence_Trained_Model/version3"
output_dir_AI_version4="/content/gdrive/My Drive/Trained_NER_Models/Artificial_Intelligence_Trained_Model/version4"


output_dir_SW="/content/gdrive/My Drive/Trained_NER_Models/Software_Engineering_Trained_Model/"

output_dir_PL_version1="/content/gdrive/My Drive/Trained_NER_Models/Programming_Language_Trained_Model/version1"
output_dir_PL_version2="/content/gdrive/My Drive/Trained_NER_Models/Programming_Language_Trained_Model/version2"
output_dir_PL_version3="/content/gdrive/My Drive/Trained_NER_Models/Programming_Language_Trained_Model/version3"


output_dir_SW_V1="/content/gdrive/My Drive/Trained_NER_Models/Software_Engineering_Trained_Model/SW_V1"

output_dir_pre_trained_model="/content/gdrive/My Drive/Trained_NER_Models/Pre_Trained_Model/"



AI_dataset_and_labels_directory="/content/gdrive/My Drive/Datasets/NER Model Dataset/Artificial Intelligence Course Dataset"
AI_dataset_text_name="Artificial_Intelligence_Course_Dataset.txt"
AI_Labels_text_name="AI_Labels.txt"


SW_dataset_and_labels_directory="/content/gdrive/My Drive/Datasets/NER Model Dataset/Software Engineering Course Dataset"
SW_dataset_text_name="sw.txt"
SW_Labels_text_name="SW_Labels.txt"

PL1_dataset_and_labels_directory="/content/gdrive/My Drive/Datasets/NER Model Dataset/Programming Language Course Dataset"
#PL1_dataset_text_name="Programming_Language_Course_Dataset.txt"
PL1_dataset_text_name="Programming_Language_Course_Dataset1.txt"
PL1_Labels_text_name="PL1_Labels.txt"




StopWordsDir="/content/gdrive/My Drive/Datasets/StopWords"
SmartStopWordsTextName="SmartStoplist.txt"


n_iter=300 #training iteration

In [ ]:
def download_pretrained_model():
  try:
    !python -m spacy download en_core_web_lg
  except:
    print("An exception occurred 1")

In [ ]:
def load_pretrained_model():
  try:
    nlp = en_core_web_lg.load()
  except:
    print("An exception occurred 2")
  

In [ ]:
def load_pretrained_model(path):
  try:
    
    print("done")
    return model.load_state_dict(torch.load(path))
  except:
    print("An exception occurred 3")

In [ ]:
def upload_corpus(dataset_and_labels_directory,dataset_text_name):
    ask_for_access_to_google_drive()
    corpus = os.path.join(dataset_and_labels_directory, dataset_text_name)
    with open(corpus, 'r',encoding='cp1252') as i_file:
      t_data = i_file.read()
    TRAIN_DATA=eval(t_data)
    return TRAIN_DATA


In [ ]:
def upload_Labels(dataset_and_labels_directory,labels_text_name):
  
    ask_for_access_to_google_drive()
    corpus = os.path.join(dataset_and_labels_directory,labels_text_name)
    with open(corpus, 'r',encoding='cp1252') as i_file:
      t_data = i_file.read()
    LABEL = t_data.split(",")
    return LABEL
  
    

In [ ]:
def Set_up_the_pipeline_and_entity_recognizer(model,LABEL):

  """Set up the pipeline and entity recognizer, and train the new entity."""
  random.seed(0)
  
  if model is not None:
      nlp = spacy.load(model) # load existing spaCy model
      print("Loaded model '%s'" % model)
  else:
      nlp = spacy.blank("en")  # create blank Language class
      print("Created blank 'en' model")

    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy

  if "ner" not in nlp.pipe_names:
      ner = nlp.create_pipe("ner")
      nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
  else:
      ner = nlp.get_pipe("ner")


  for en in LABEL:
    
    ner.add_label(en) # add new entity label to entity recognizer

    # Adding extraneous labels shouldn't mess anything up
    #ner.add_label("VEGETABLE")
  if model is None:
      optimizer = nlp.begin_training()
  else:
      optimizer = nlp.resume_training()
  move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
  pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
  return(nlp, other_pipes,move_names,optimizer)
  

In [ ]:
def train_model(TRAIN_DATA,nlp,other_pipes,optimizer):

  start_time = datetime.now() 
  
  training_losses=list()
  with nlp.disable_pipes(*other_pipes):  # only train NER
    sizes = compounding(1.0, 4.0, 1.001)
      # batch up the examples using spaCy's minibatch
    count = 0
    for itn in range(n_iter):
        small_list=list()
        count = count+1
        random.shuffle(TRAIN_DATA)
        batches = minibatch(TRAIN_DATA, size=sizes)
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
        print(count," : ","Losses", losses)
        small_list.append(count)
        small_list.append(losses)
        training_losses.append(small_list)
 
  model_training_time =format(datetime.now() - start_time)
  print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))
  return nlp,training_losses,model_training_time
  

In [ ]:
def ask_for_access_to_google_drive():
  try:
    drive.mount('/content/gdrive')
  except:
    print("An exception occurred 8")

In [ ]:
def save_trained_model(new_model_name, nlp, output_dir):
  try:
    ask_for_access_to_google_drive()
    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        #------------------------------------------------------------->askfor rename model
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
        return 1 #------------------------------------------------>change it
  except:
    print("An exception occurred 9")

In [ ]:
def load_trained_model(output_dir):
  try:
    ask_for_access_to_google_drive()
    output_dir = Path(output_dir)
    print("Loading from", output_dir)
    nlp = spacy.load(output_dir)
    return nlp #return loaded AI trained model
  except:
    print("An exception occurred 10")

In [ ]:
def test_model_after_loading(sentence,nlp):
    # test the saved model
    # Check the classes have loaded back consistently
    #assert nlp.get_pipe("ner").move_names == move_names
    doc = nlp(sentence)
    return doc

In [ ]:
def test_model_after_training(sentence,nlp):

    doc = nlp(sentence)
    return doc


In [ ]:
def print_entities(sentence):
  try:
    for ent in sentence.ents:
      print(sentence)
      print("Label: ",ent.label_," Keyword: ", ent.text)
  except:
    print("An exception occurred 13")

In [ ]:
def print_entities1(doc):
  try:
    sentences = [x for x in doc.sents]
    for sentence in sentences:
      print(sentence)
      for ent in sentence.ents:
        print("Label: ",ent.label_," Keyword: ", ent.text)
      print("")
  except:
    print("An exception occurred 13")

In [ ]:
def print_entities_by_displacy(sentence):
  try:
    displacy.render(sentence, jupyter=True, style='ent')

  except:
    print("An exception occurred 14")

In [ ]:

def print_entities_by_displacy1(doc):
  try:
    displacy.render(doc, jupyter=True, style='ent')
  except:
    print("An exception occurred 14")

In [ ]:
def Set_a_name_to_the_new_model():
  try:
    new_model_name = input("Enter A name for the new model: ")
  except:
    print("An exception occurred 15")

## **Calling**

In [ ]:
#uploaded = files.upload() 

In [ ]:
#with open('Programming_Language_Course_Dataset.txt', 'r') as i_file:
#    t_data = i_file.read()
##print(t_data)    
#TRAIN_DATA=eval(t_data)

In [ ]:
##with open(corpus, 'r',encoding='cp1252') as i_file:
##  t_data = i_file.read()
##TRAIN_DATA=eval(t_data)

In [ ]:
PL1_dataset_and_labels_directory="/content/gdrive/My Drive/Datasets/NER Model Dataset/Programming Language Course Dataset"
PL1_dataset_text_name="Programming_Language_Course_Dataset1.txt"
PL1_Labels_text_name="PL1_Labels.txt"

In [ ]:
#connect to drive once in session
TRAIN_DATA=upload_corpus(PL1_dataset_and_labels_directory , PL1_dataset_text_name)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
An exception occurred 8


FileNotFoundError: ignored

In [ ]:
LABEL=upload_Labels(PL1_dataset_and_labels_directory , PL1_Labels_text_name)

In [ ]:
#TRAIN_DATA

In [ ]:
print(LABEL)

['PL1']


In [ ]:
#test_text = "all the skills needed for the Turing Test,imitation game. reem is a good person. meshmesh is a good cat. he eats alot of dry food. GDP in developing countries such as Vietnam will continue growing at a high rate. Here is how you can keep your car and other vehicles clean. Here is how you can keep your car or other vehicles clean. Eight people, including two children, were injured in the explosion. A healthy eating pattern includes fruits, especially whole fruits. mom answers the phone, and the table is broken by marwan. today will be a good day. the garden was watered by the man. how does k-means algorithm work? and what about talking about data science. all the skills needed for the Turing Test,imitation game. alaa was fantastic in microsoft.  the environment does not change with the passage of time,but the agent's performance score does. The introduction of spreadsheets enabled business users to create simple logic on data structured in rows and columns and create their own analyses of business problems. Database administrator training is not required to create spreadsheets: They can be set up to do many things quickly and independently of information technology (IT) groups. Spreadsheets are easy to share, and end users have control over the logic involved. However, their proliferation can result in “many versions of the truth.” In other words, it can be challenging to determine if a particular user has the most relevant version of a spreadsheet, with the most current data and logic in it. Moreover, if a laptop is lost or a file becomes corrupted, the data and logic within the spreadsheet could be lost. This is an ongoing challenge because spreadsheet programs such as Microsoft Excel still run on many computers worldwide. With the proliferation of data islands (or spreadmarts), the need to centralize the data is more pressing than ever. As data needs grew, so did more scalable data warehousing solutions. supervised learning is a type of machine learning. there is a lot of supervised learning algorithms like support vector machine algorithm , knn algorithm , logistic regression , and many more. "

In [ ]:
#test_text = "how does k-means algorithm work? and what about talking about data science. all the skills needed for the Turing Test,imitation game. alaa was fantastic in microsoft.  the environment does not change with the passage of time,but the agent's performance score does. The introduction of spreadsheets enabled business users to create simple logic on data structured in rows and columns and create their own analyses of business problems. Database administrator training is not required to create spreadsheets: They can be set up to do many things quickly and independently of information technology (IT) groups. Spreadsheets are easy to share, and end users have control over the logic involved. However, their proliferation can result in “many versions of the truth.” In other words, it can be challenging to determine if a particular user has the most relevant version of a spreadsheet, with the most current data and logic in it. Moreover, if a laptop is lost or a file becomes corrupted, the data and logic within the spreadsheet could be lost. This is an ongoing challenge because spreadsheet programs such as Microsoft Excel still run on many computers worldwide. With the proliferation of data islands (or spreadmarts), the need to centralize the data is more pressing than ever. As data needs grew, so did more scalable data warehousing solutions. These technologies"


In [ ]:

test_text = "Data Science is the science which uses computer science, statistics and machine learning, visualization and human computer interactions to collect , clean , integrate , analyse , visualize , interact with data to create data products. GDP in developing countries such as Vietnam will continue growing at a high rate. Here is how you can keep your car and other vehicles clean. Here is how you can keep your car or other vehicles clean. Eight people, including two children, were injured in the explosion. A healthy eating pattern includes fruits, especially whole fruits. We are going to watch a movie tonight. The novel was read by Mom in one day."

In [ ]:
nlp,other_pipes,move_names,optimizer=Set_up_the_pipeline_and_entity_recognizer(model,LABEL) 

Loaded model 'en_core_web_sm'


In [ ]:
PL1_dataset_and_labels_directory="/content/gdrive/My Drive/Datasets/NER Model Dataset/Programming Language Course Dataset"
PL1_dataset_text_name="pl_text.txt"
PL1_Labels_text_name="PL1_Labels.txt"

#connect to drive once in session
TRAIN_DATA=upload_corpus(PL1_dataset_and_labels_directory , PL1_dataset_text_name)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
TRAIN_DATA

[('why learn c programming?c helps you to understand the internal architecture of a computer, how computer stores and retrieves information.after learning c, it will be much easier to learn other programming languages like java, python, etc.opportunity to work on open source projects.',
  {'entities': [(12, 23, 'PL1'),
    (193, 214, 'PL1'),
    (220, 224, 'PL1'),
    (226, 232, 'PL1')]}),
 ('some of the largest open-source projects such as linux kernel, python interpreter, sqlite database, etc. are written in c programming.how to learn c',
  {'entities': [(83, 98, 'PL1'), (122, 133, 'PL1')]}),
 ('visit official c programming documentation.write a lot of c programming code - the only way you can learn programming is by writing a lot of code.',
  {'entities': [(60, 76, 'PL1'), (141, 145, 'PL1')]}),
 ('by design, c provides constructs that map efficiently to typical machine instructions and has found lasting use in applications previously coded in assembly language.',
  {'entities': [(22

In [ ]:
LABEL=upload_Labels(PL1_dataset_and_labels_directory , PL1_Labels_text_name)

nlp,other_pipes,move_names,optimizer=Set_up_the_pipeline_and_entity_recognizer(model,LABEL) 
 
nlp,training_losses,model_training_time=train_model(TRAIN_DATA,nlp,other_pipes,optimizer)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Loaded model 'en_core_web_sm'
1  :  Losses {'ner': 5153.29266984877}
2  :  Losses {'ner': 5079.291827362578}
3  :  Losses {'ner': 4963.170859398142}
4  :  Losses {'ner': 4910.843218260445}
5  :  Losses {'ner': 4845.955585864838}
6  :  Losses {'ner': 4923.382914362475}
7  :  Losses {'ner': 4824.30200445652}
8  :  Losses {'ner': 4750.37809048593}
9  :  Losses {'ner': 4813.342358569193}
10  :  Losses {'ner': 4811.607479065657}
11  :  Losses {'ner': 4687.225244492292}
12  :  Losses {'ner': 4577.768663004041}
13  :  Losses {'ner': 4703.433012437075}
14  :  Losses {'ner': 4625.067056834698}
15  :  Losses {'ner': 4660.668163612485}
16  :  Losses {'ner': 4686.297447562218}
17  :  Losses {'ner': 4604.0366446226835}
18  :  Losses {'ner': 4661.50434319675}
19  :  Losses {'ner': 4543.238852113485}
20  :  Losses {'ner': 4550.964843824506}
21  :  Losses {'ner': 4599.6413

In [ ]:
PL1_model_name="Programming_Language_1_Model"

In [ ]:
s=save_trained_model(PL1_model_name,nlp,output_dir_PL_version3)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Saved model to /content/gdrive/My Drive/Trained_NER_Models/Programming_Language_Trained_Model/version3


In [ ]:
nlp=load_trained_model(output_dir_SW_V1)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Loading from /content/gdrive/My Drive/Trained_NER_Models/Software_Engineering_Trained_Model/SW_V1


In [ ]:
def break_document_into_sentences(doc):
  try:
    sentences = [x for x in doc.sents]
    return sentences
  except:
    print("An exception occurred 20")

In [ ]:
def return_sentences_with_entities(doc):
  try:
    list_of_sentences_with_entities=list()
    sentences = break_document_into_sentences(doc)
    for sentence in sentences:
      if (sentence.ents != []):
        list_of_sentences_with_entities.append(sentence)   
    return list_of_sentences_with_entities
  except:
    print("An exception occurred 21")

In [ ]:
def is_sentence_has_entities(sentence):
  try:
    if (sentence.ents != []):
      return 1
    else:
      return 0
  except:
    print("An exception occurred 21")

In [ ]:
def return_sentence_and_its_entities(sentence):
  try:
    list_sentence_entity_label=list()
    list_of_entities_and_its_label=list()
    list_sentence_entity_label.append(sentence)
    for ent in sentence.ents:
      small_list=list()
      small_list.append(ent.text)
      small_list.append(ent.label_)
      list_of_entities_and_its_label.append(small_list)
    list_sentence_entity_label.append(list_of_entities_and_its_label)
    return list_sentence_entity_label
  except:
    print("An exception occurred 13")

In [ ]:
def return_sentence_and_its_entities1(sentence):
  try:
    
    keyword='NER' #it holds the extracted keyword
    label=''
    Name='NER'#it holds the name of model

    dic[sentence]={'Keyword':Name, 'Label':Name}


    #list_sentence_entity_label=list()
    list_of_entities_and_its_label=list()
    #list_sentence_entity_label.append(sentence)
    for ent in sentence.ents:
      small_list=list()
      small_list.append(ent.text)
      small_list.append(ent.label_)
      list_of_entities_and_its_label.append(small_list)
    #list_sentence_entity_label.append(list_of_entities_and_its_label)
    return #list_sentence_entity_label
  except:
    print("An exception occurred 13")

In [ ]:
def return_sentence_and_its_entities1(sentence):
  try:
    list_sentence_entity_label=list()
    list_of_entities_and_its_label=list()
    list_sentence_entity_label.append(sentence)
    for ent in sentence.ents:
      small_list=list()
      small_list.append(ent.text)
      small_list.append(ent.label_)
      list_of_entities_and_its_label.append(small_list)
    list_sentence_entity_label.append(list_of_entities_and_its_label)
    return list_sentence_entity_label
  except:
    print("An exception occurred 13")

In [ ]:
doc=test_model_after_training(test_text,nlp)

In [ ]:
sentences_with_entities = return_sentences_with_entities(doc)

In [ ]:
for sentence in sentences_with_entities:
  print_entities_by_displacy(sentence)

In [ ]:
sentences_with_entities

[]

In [ ]:
def test_NER_Model(test_text,nlp):
  doc= test_model_after_loading(test_text,nlp)
  sentences_with_entities = return_sentences_with_entities(doc)
  large_list=list()
  for sentence in sentences_with_entities:
    large_list.append(return_sentence_and_its_entities(sentence))
  return large_list

In [ ]:
def test_NER_Model1(sentence,nlp):
  boolean_var = is_sentence_has_entities(sentence)
  if (boolean_var==1):
    result_list = return_sentence_and_its_entities(sentence)
    return result_list
  else:
    return 0

In [ ]:
large_list=list()
for sentence in sentences_with_entities:
  large_list.append(return_sentence_and_its_entities(sentence))

In [ ]:
for e in large_list:
 print(e)
 print("")
 print("")

In [ ]:
result=test(test_text,nlp)

NameError: ignored

# **-------------------------------------------------------------------------------------------------**

# pattern

**spaCy’s Rule-Based Matching**

Rule-Based Matching
Unlike regular expression’s fixed pattern matching, this helps us match token, phrases and entities of words and sentences according to some pre-set patterns along with the features such as parts-of-speech, entity types, dependency parsing, lemmatization and many more. Not only these, but this also supports regular expression patterns which makes the rule-based matching approach of spaCy has an upper hand among other NLP software libraries available today.



We can infer that “Gelidium” is a type of “red algae” just by looking at the structure of the sentence.

In linguistics terms, we will call “red algae” as Hypernym and “Gelidium” as its Hyponym.

In [ ]:
#patterns Packages
import re 
import string 
import nltk 
#import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
#from spacy import displacy 

pd.set_option('display.max_colwidth', 200)

In [ ]:

#define the pattern
pattern1 = [{'DEP':'amod', 'OP':"?"}, # adjectival modifier
           {'POS':'NOUN'},
           {'LOWER': 'such'},
           {'LOWER': 'as'},
           {'POS': 'PROPN'}]



#define the pattern 
pattern2 = [{'DEP':'amod', 'OP':"?"}, 
           {'POS':'NOUN'}, 
           {'LOWER': 'and', 'OP':"?"}, 
           {'LOWER': 'or', 'OP':"?"}, 
           {'LOWER': 'other'}, 
           {'POS': 'NOUN'}] 
      
#define the pattern 
pattern3 = [{'DEP':'nummod','OP':"?"}, # numeric modifier 
           {'DEP':'amod','OP':"?"}, # adjectival modifier 
           {'POS':'NOUN'}, 
           {'IS_PUNCT': True}, 
           {'LOWER': 'including'}, 
           {'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}] 

#define the pattern 
pattern4 = [{'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}, 
           {'IS_PUNCT':True}, 
           {'LOWER': 'especially'}, 
           {'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}] 

In [ ]:
# sample text 
text = "reem is a good person. meshmesh is a good cat. he eats alot of dry food. GDP in developing countries such as Vietnam will continue growing at a high rate. Here is how you can keep your car and other vehicles clean. Here is how you can keep your car or other vehicles clean. Eight people, including two children, were injured in the explosion. A healthy eating pattern includes fruits, especially whole fruits.  " 
#tessst="hello cutie"
# create a spaCy object 
#doc = nlp(text)

In [ ]:
# load spaCy model
#nlp = spacy.load("en_core_web_sm")

In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 

In [ ]:
# Add match ID "matching_1" with no callback and one pattern

matcher.add("pattern_1", None, pattern1) #Pattern: X such as Y , ‘countries such as Vietnam’
matcher.add("pattern_2", None, pattern2)
matcher.add("pattern_3", None, pattern3)
matcher.add("pattern_4", None, pattern4)

In [ ]:
def testPatterns(test_text,nlp, matcher):
    #doc = nlp(test_text)
    sentences_with_patterns = return_sentences_with_patterns(test_text,matcher)  #wchich found in it an aintites
    #print(sentences_with_patterns)
    large_list=list()
    for sentence in sentences_with_patterns:
        large_list.append(return_sentence_and_its_patterns(sentence,nlp,matcher))
    return large_list

In [ ]:
def testPatterns1(sentence,nlp, matcher):
    boolean_var = is_sentence_has_patterns_matched(sentence,nlp, matcher)  #wchich found in it an aintites
    if (boolean_var == 1): #it has 
      result_list = return_sentence_and_its_patterns(sentence, nlp, matcher)
      return result_list
    else:
      return 0

In [ ]:
def return_sentences_with_patterns(doc,matcher):
    list_of_sentences_with_patterns=list()
    doc=nlp(doc)
    sentences = break_document_into_sentences(doc)
    #print(sentences)
    for sentence in sentences:
        doc=nlp(str(sentence))
        matches=matcher(doc)
        if (matches != []):
            list_of_sentences_with_patterns.append(sentence[0:-1])   
    return list_of_sentences_with_patterns

In [ ]:
def is_sentence_has_patterns_matched(sentence,nlp,matcher):
  doc=nlp(str(sentence))
  matches=matcher(doc)
  if (matches != []):  
    return 1

In [ ]:
def return_sentence_and_its_patterns(sentence,nlp,matcher):
        doc=nlp(str(sentence))
        matches = matcher(doc)
        list_sentence_and_pattern_config=list()
        list_of_matchesID_patternID_span_start_end=list()
        list_sentence_and_pattern_config.append(str(sentence))
        for match_id, start, end in matches:
            small_list=list()
            string_id = nlp.vocab.strings[match_id]  # Get string representation
            span = doc[start:end]  # The matched span
            small_list.append(match_id)
            small_list.append(string_id)
            small_list.append(start)
            small_list.append(end)
            small_list.append(span.text)

            list_of_matchesID_patternID_span_start_end.append(small_list)
        list_sentence_and_pattern_config.append(list_of_matchesID_patternID_span_start_end)
        return list_sentence_and_pattern_config

In [ ]:
l=testPatterns(text,nlp,matcher)

In [ ]:
for i in l:
  print(i)
  print("")
  print("")

['GDP in developing countries such as Vietnam will continue growing at a high rate', [[13888292881017009137, 'pattern_1', 2, 7, 'developing countries such as Vietnam'], [13888292881017009137, 'pattern_1', 3, 7, 'countries such as Vietnam']]]


['Here is how you can keep your car and other vehicles clean', [[15719126060659784350, 'pattern_2', 7, 11, 'car and other vehicles']]]


['Here is how you can keep your car or other vehicles clean', [[15719126060659784350, 'pattern_2', 7, 11, 'car or other vehicles']]]


['Eight people, including two children, were injured in the explosion', [[13375652922493068680, 'pattern_3', 0, 6, 'Eight people, including two children'], [13375652922493068680, 'pattern_3', 1, 6, 'people, including two children']]]


['A healthy eating pattern includes fruits, especially whole fruits.', [[9514930324123629828, 'pattern_4', 5, 10, 'fruits, especially whole fruits']]]




In [ ]:
def testSubTreeMAtcher(test_text,nlp):
    #doc = nlp(test_text)
    sentences_with_SubTreeMAtcher = return_sentences_with_SubTreeMAtcher(test_text,nlp)  #wchich found in it an aintites
    large_list=list()
    for sentence in sentences_with_SubTreeMAtcher:
        large_list.append(return_sentence_and_its_SubTreeMAtcher(sentence,nlp))
    return large_list

In [ ]:
def testSubTreeMAtcher1(sentence,nlp):
    #doc = nlp(test_text)
    boolean_var = is_sentence_has_SubTreeMatching(sentence)  #wchich found in it an aintites
    if (boolean_var==1):
      result_list = return_sentence_and_its_SubTreeMAtcher(sentence,nlp)
      return result_list
    else:
      return 0

In [ ]:
#g=testSubTreeMAtcher(text5,nlp)

In [ ]:
#g

In [ ]:
def return_sentences_with_SubTreeMAtcher(doc,nlp):
  substring1=","
  list_of_sentences_with_SubTreeMAtcher=list()
  doc=nlp(doc)
  sentences = break_document_into_sentences(doc)
  for sentence in sentences:
      sentence_has_SubTreeMAtcher=0
      if (substring1 in str(sentence)):
        list_of_sentences = str(sentence).split(",")
        for each_sentence in list_of_sentences:
            if (sentence_has_SubTreeMAtcher==0):
              doc=nlp(str(each_sentence))
              x,y,flag =new_subtree_matcher(doc)
              if ((x != '') | (y !='')):
                sentence_has_SubTreeMAtcher=1
                list_of_sentences_with_SubTreeMAtcher.append(str(sentence))
      else:
        doc=nlp(str(sentence)) #change it to each sentence if something wrong
        x,y,flag =new_subtree_matcher(doc)
        if ((x != '') | (y !='')):
          list_of_sentences_with_SubTreeMAtcher.append(str(sentence))
  return list_of_sentences_with_SubTreeMAtcher

In [ ]:
def is_sentence_has_SubTreeMatching(sentence):
  substring1=","
  sentence_has_SubTreeMAtcher=0
  if (substring1 in str(sentence)):
    list_of_sentences = str(sentence).split(",")
    for each_sentence in list_of_sentences:
        if (sentence_has_SubTreeMAtcher==0): #if it doesnt have matching yet
          doc=nlp(str(each_sentence))
          x,y,flag =new_subtree_matcher(doc)
          if ((x != '') | (y !='')):
            sentence_has_SubTreeMAtcher=1 
          else:
            return 0
        else: 
          return 1
  else:
    doc=nlp(str(sentence))
    x,y,flag =new_subtree_matcher(doc)
    if ((x != '') | (y !='')):
      return 1
    else:
      return 0

In [ ]:
def return_sentence_and_its_SubTreeMAtcher(sentence,nlp):
  substring1=","
  list_sentence_and_SubTreeMAtcher_config=list()
  list_of_sub_sentence_x_y_flag=list()
  list_sentence_and_SubTreeMAtcher_config.append(sentence)
  if (substring1 in str(sentence)):
    list_of_sentences = str(sentence).split(",")
    for each_sub_sentence in list_of_sentences:
      small_list=list()
      doc=nlp(str(each_sub_sentence))
      x,y,flag =new_subtree_matcher(doc)
      small_list.append(x)
      small_list.append(y)
      small_list.append(flag)
      small_list.append(each_sub_sentence)
      list_of_sub_sentence_x_y_flag.append(small_list)
  else:
      small_list=list()
      doc=nlp(str(sentence))
      x,y,flag =new_subtree_matcher(doc)
      small_list.append(x)
      small_list.append(y)
      small_list.append(flag)
      list_of_sub_sentence_x_y_flag.append(small_list)

  list_sentence_and_SubTreeMAtcher_config.append(list_of_sub_sentence_x_y_flag)
  return list_sentence_and_SubTreeMAtcher_config

In [ ]:
def new_subtree_matcher(doc):
  subjpass = 0

  for i,tok in enumerate(doc):
    # find dependency tag that contains the text "subjpass"    
    if tok.dep_.find("subjpass") == True:
      subjpass = 1

  x = ''
  y = ''
  flag=''
  # if subjpass == 1 then sentence is passive
  if subjpass == 1:
    flag=flag+"passive sentence"
    for i,tok in enumerate(doc):
      if tok.dep_.find("subjpass") == True:
        if (y == ''):
          y = tok.text

      if tok.dep_.endswith("obj") == True:
        if (x == ''):
          x = tok.text
  
  # if subjpass == 0 then sentence is not passive
  else:
    flag=flag+"not passive sentence"
    for i,tok in enumerate(doc):
      if tok.dep_.endswith("subj") == True:
        if (x == ''):
          x = tok.text

      if tok.dep_.endswith("obj") == True:
        if (y == ''):
          y = tok.text

  return x,y,flag

## **----------------------------------------------------------------------------------**

## **RAKE**

In [ ]:
!pip install rake-nltk
!pip install python-rake==1.4.4
import RAKE

stop_dir = os.path.join(StopWordsDir,SmartStopWordsTextName)
rake_object = RAKE.Rake(stop_dir)

In [ ]:
stop_dir = os.path.join(StopWordsDir,SmartStopWordsTextName)
rake_object = RAKE.Rake(stop_dir)

In [ ]:
test_text = "all the skills needed for the Turing Test,imitation game. reem is a good person. meshmesh is a good cat. he eats alot of dry food. GDP in developing countries such as Vietnam will continue growing at a high rate. Here is how you can keep your car and other vehicles clean. Here is how you can keep your car or other vehicles clean. Eight people, including two children, were injured in the explosion. A healthy eating pattern includes fruits, especially whole fruits. mom answers the phone, and the table is broken by marwan. today will be a good day. the garden was watered by the man. how does k-means algorithm work? and what about talking about data science. all the skills needed for the Turing Test,imitation game. alaa was fantastic in microsoft.  the environment does not change with the passage of time,but the agent's performance score does. The introduction of spreadsheets enabled business users to create simple logic on data structured in rows and columns and create their own analyses of business problems. Database administrator training is not required to create spreadsheets: They can be set up to do many things quickly and independently of information technology (IT) groups. Spreadsheets are easy to share, and end users have control over the logic involved. However, their proliferation can result in “many versions of the truth.” In other words, it can be challenging to determine if a particular user has the most relevant version of a spreadsheet, with the most current data and logic in it. Moreover, if a laptop is lost or a file becomes corrupted, the data and logic within the spreadsheet could be lost. This is an ongoing challenge because spreadsheet programs such as Microsoft Excel still run on many computers worldwide. With the proliferation of data islands (or spreadmarts), the need to centralize the data is more pressing than ever. As data needs grew, so did more scalable data warehousing solutions. supervised learning is a type of machine learning. there is a lot of supervised learning algorithms like support vector machine algorithm , knn algorithm , logistic regression , and many more. "

In [ ]:
from rake_nltk import Rake
import operator
import RAKE
import nltk
import os

In [ ]:
def test_Rake_keyword_extraction(test_text,nlp):
    #doc = nlp(test_text)
    sentences_with_Rake_keyword_extraction = return_sentences_with_Rake_keyword_extraction(test_text,nlp)  #wchich found in it an aintites
    large_list=list()
    for sentence in sentences_with_Rake_keyword_extraction:
        large_list.append(return_sentence_and_its_Rake_keyword_extraction(sentence))
    return large_list

In [ ]:
def test_Rake_keyword_extraction1(sentence,nlp,rake_object):
    boolean_var = is_it_has_Rake_keyword_extraction(sentence,rake_object)  #wchich found in it an aintites
    if (boolean_var == 1): #it has 
      result_list = return_sentence_and_its_Rake_keyword_extraction(sentence,rake_object)
      return result_list
    else:
      return 0

In [ ]:
def return_sentences_with_Rake_keyword_extraction(doc,nlp,rake_object):
  list_of_sentences_with_Rake_keyword_extraction=list()
  doc=nlp(doc)
  sentences = break_document_into_sentences(doc)
  for sentence in sentences:
      keywords = rake_object.run(str(sentence))
      if (keywords!=[]):
        if (keywords[0][1]!=0):
          list_of_sentences_with_Rake_keyword_extraction.append(str(sentence))

  return list_of_sentences_with_Rake_keyword_extraction

In [ ]:
def is_it_has_Rake_keyword_extraction(sentence,rake_object):
  keywords = rake_object.run(str(sentence))
  if (keywords!=[]):
    if (keywords[0][1]!=0):
      return 1
    else:
      return 0
  else:
    return 0

In [ ]:
def return_sentence_and_its_Rake_keyword_extraction(sentence,rake_object):
  list_sentence_and_Rake_keyword_extraction_config=list()
  list_of_highestKeyword_Score=list()
  sentence=str(sentence)
  list_sentence_and_Rake_keyword_extraction_config.append(sentence)
  keywords = rake_object.run(sentence)
  c=3
  for each_keyword in keywords:
    if (c==0):
      break
    small_list=list()
    small_list.append(each_keyword[0])
    small_list.append(each_keyword[1])
    list_of_highestKeyword_Score.append(small_list)
    c=c-1

  list_sentence_and_Rake_keyword_extraction_config.append(list_of_highestKeyword_Score)
  return list_sentence_and_Rake_keyword_extraction_config

# **mix all up --------------------------------------------------------------**

In [ ]:
def create_complete_question_function(i):
  switcher={
          0:test_NER_Model(test_text,nlp),
          1:testPatterns(test_text,nlp,matcher),
          2:testSubTreeMAtcher(test_text,nlp),
          3:test_Rake_keyword_extraction(test_text,nlp),
          4:'Thursday',
          5:'Friday',
          6:'Saturday'
        }
  return switcher.get(i,"Invalid day of week")

In [ ]:
def return_NER_keywords_list(NER_list):
  keywords_list=list()
  #NER_list[0] #holds sentence , and we dont need it
  for eachList in NER_list[1]:
    keywords_list.append(eachList[0])
  return keywords_list

In [ ]:
def return_patterns_keywords_list(patterns_list):
  keywords_list=list()
  #patterns_list[0] #holds sentence , and we dont need it
  for eachList in patterns_list[1]:
    text=eachList[4]
    if (eachList[1]=='pattern_1'):
      list_of_keywords=text.split("such as")
      for i in list_of_keywords:
        keywords_list.append(i)

    if (eachList[1]=='pattern_2'):
      if ("and other" in text):
        list_of_keywords=text.split("such as")
        for i in list_of_keywords:
          keywords_list.append(i)
      
      if ("or other" in text):
        list_of_keywords=text.split("or as")
        for i in list_of_keywords:
          keywords_list.append(i)

    if (eachList[1]=='pattern_3'):
      list_of_keywords=text.split("including")
      for i in list_of_keywords:
        keywords_list.append(i)

    if (eachList[1]=='pattern_4'):
      list_of_keywords=text.split("especially")
      for i in list_of_keywords:
        keywords_list.append(i)
      
  return keywords_list

In [ ]:
def return_SubTreeMatcher_keywords_list(SubTreeMatcher_keywords_list):
  keywords_list=list()
  for eachList in SubTreeMatcher_keywords_list[1]:
    keywords_list.append(eachList[0]) #subject
    keywords_list.append(eachList[1]) #object
  return keywords_list

In [ ]:
def return_RAKE_keywords_list(RAKE_keywords_list):
  keywords_list=list()
  for eachList in RAKE_keywords_list[1]:
    keywords_list.append(eachList[0]) #subject
  return keywords_list

# **TEST ALL ----------------------------------------------------**

In [ ]:
nlp=load_trained_model(output_dir_AI_version4)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Loading from /content/gdrive/My Drive/Trained_NER_Models/Artificial_Intelligence_Trained_Model/version4


In [ ]:
matcher = Matcher(nlp.vocab) 

In [ ]:
doc=test_model_after_loading(test_text,nlp)

In [ ]:
sentences = break_document_into_sentences(doc)

In [ ]:
def create_complete_question_function1(sentences, nlp, rake_object):

  dic={}
  Approach1='NER' 
  Approach2='Pattern'
  Approach3='SubTreeMatcher'
  Approach4='Rake'

  for sentence in sentences:
    
    sentence_in_str_format=str(sentence)
    list1 = test_NER_Model1(sentence,nlp)
    list2 = testPatterns1(sentence,nlp,matcher)
    list3 = testSubTreeMAtcher1(sentence,nlp)
    list4 = test_Rake_keyword_extraction1(sentence,nlp,rake_object)

    if(list1 !=0):
      NER_keywords_list = return_NER_keywords_list(list1)
      list1_keywords=NER_keywords_list
    else:
      list1_keywords=0
    
    if(list2 !=0):
      pattern_keywords_list = return_patterns_keywords_list(list2)
      list2_keywords=pattern_keywords_list
    else:
      list2_keywords=0

    
    if(list3 !=0):
      SubTreeMatcher_keywords_list = return_SubTreeMatcher_keywords_list(list3)
      list3_keywords=SubTreeMatcher_keywords_list
    else:
      list3_keywords=0

    if(list4 !=0):
      RAKE_keywords_list = return_RAKE_keywords_list(list4)
      list4_keywords=RAKE_keywords_list
    else:
      list4_keywords=0

    dic[sentence_in_str_format]={Approach1:list1_keywords, Approach2:list2_keywords, Approach3:list3_keywords, Approach4:list4_keywords}

  return dic

In [ ]:
def put_blank_Space_in_NER_Sentences(sentence,keyword):
  text=str(sentence)
  sentence_with_blank_Space = text.replace(keyword, " _________ ")
  return sentence_with_blank_Space

In [ ]:
put_blank_Space_in_NER_Sentences("reem loves cats.","reem ")


' _________ loves cats.'

In [ ]:
dic2 = create_complete_question_function1(sentences, nlp,rake_object)

In [ ]:

for sentence in sentences:
  print("=============================================================================================")
  print(sentence)
  print("=============================================================================================")

  print("1- NER Approach *****************************************************************************")
  if (dic2[str(sentence)]['NER']!=0):
    print(dic2[str(sentence)]['NER'])
  print("")

  print("2- Pattern Approach *************************************************************************")
  if (dic2[str(sentence)]['Pattern']!=0):
    print(dic2[str(sentence)]['Pattern'])
  print("")

  print("3- SubTreeMatcher Approach ******************************************************************")
  if (dic2[str(sentence)]['SubTreeMatcher']!=0):
    print(dic2[str(sentence)]['SubTreeMatcher'])
  print("")

  print("4- RAKE Approach ****************************************************************************")
  if (dic2[str(sentence)]['Rake']!=0):
    print(dic2[str(sentence)]['Rake'])

  print("")
  print("")

all the skills needed for the Turing Test,imitation game.
1- NER Approach *****************************************************************************
['Turing Test', 'imitation game']

2- Pattern Approach *************************************************************************

3- SubTreeMatcher Approach ******************************************************************
['skills', 'Test', '', '']

4- RAKE Approach ****************************************************************************
['skills needed', 'turing test', 'imitation game']


reem is a good person.
1- NER Approach *****************************************************************************

2- Pattern Approach *************************************************************************

3- SubTreeMatcher Approach ******************************************************************
['reem', '']

4- RAKE Approach ****************************************************************************
['good person', 'reem']


meshmesh

In [ ]:
dic3 = create_complete_question_function1(sentences, nlp,rake_object)
print("=============================================================================================")
print("1- NER Approach *****************************************************************************")
print("=============================================================================================")
list_of_sentences_with_blank_space=list()
for sentence in sentences:
  if (dic3[str(sentence)]['NER']!=0):
    
    #print(sentence)
    #print(dic2[str(sentence)]['NER'])
    z= put_blank_Space_in_NER_Sentences( str(sentence) ,dic2[str(sentence)]['NER'][0])
    list_of_sentences_with_blank_space.append(z)
    print(z)
    print("")
  else:
    continue

  #print("2- Pattern Approach *************************************************************************")
  #if (dic2[str(sentence)]['Pattern']!=0):
   # print(dic2[str(sentence)]['Pattern'])
  #print("")

  #print("3- SubTreeMatcher Approach ******************************************************************")
  #if (dic2[str(sentence)]['SubTreeMatcher']!=0):
  #  print(dic2[str(sentence)]['SubTreeMatcher'])
  #print("")

  #print("4- RAKE Approach ****************************************************************************")
  #if (dic2[str(sentence)]['Rake']!=0):
  #  print(dic2[str(sentence)]['Rake'])

1- NER Approach *****************************************************************************
all the skills needed for the  _________ ,imitation game.

GDP in developing countries such as  _________  will continue growing at a high rate.

all the skills needed for the  _________ ,imitation game.

there is a lot of supervised learning algorithms like support vector machine algorithm , knn algorithm , logistic  _________  , and many more.



In [ ]:
create_complete_question_function(0)

[all the skills needed for the Turing Test,imitation game., reem is a good person., meshmesh is a good cat., he eats alot of dry food., GDP in developing countries such as Vietnam will continue growing at a high rate., Here is how you can keep your car and other vehicles clean., Here is how you can keep your car or other vehicles clean., Eight people, including two children, were injured in the explosion., A healthy eating pattern includes fruits, especially whole fruits., mom answers the phone, and the table is broken by marwan., today will be a good day., the garden was watered by the man., how does k-means algorithm work?, and what about talking about data science., all the skills needed for the Turing Test,imitation game., alaa was fantastic in microsoft.  , the environment does not change with the passage of time,but the agent's performance score does., The introduction of spreadsheets enabled business users to create simple logic on data structured in rows and columns and create 

[[all the skills needed for the Turing Test,imitation game.,
  [['Turing Test', 'experiment'], ['imitation game', 'actingHumanly']]],
 [all the skills needed for the Turing Test,imitation game.,
  [['Turing Test', 'experiment'], ['imitation game', 'actingHumanly']]],
 [there is a lot of supervised learning algorithms like support vector machine algorithm , knn algorithm , logistic regression , and many more.,
  [['regression', 'Supervised learning']]]]

In [ ]:
create_complete_question_function(1)

[['GDP in developing countries such as Vietnam will continue growing at a high rate',
  [[13888292881017009137,
    'pattern_1',
    2,
    7,
    'developing countries such as Vietnam'],
   [13888292881017009137, 'pattern_1', 3, 7, 'countries such as Vietnam']]],
 ['Here is how you can keep your car and other vehicles clean',
  [[15719126060659784350, 'pattern_2', 7, 11, 'car and other vehicles']]],
 ['Here is how you can keep your car or other vehicles clean',
  [[15719126060659784350, 'pattern_2', 7, 11, 'car or other vehicles']]],
 ['Eight people, including two children, were injured in the explosion',
  [[13375652922493068680,
    'pattern_3',
    0,
    6,
    'Eight people, including two children'],
   [13375652922493068680,
    'pattern_3',
    1,
    6,
    'people, including two children']]],
 ['A healthy eating pattern includes fruits, especially whole fruits',
  [[9514930324123629828,
    'pattern_4',
    5,
    10,
    'fruits, especially whole fruits']]],
 ['This is an on

In [ ]:
create_complete_question_function(2)

[['all the skills needed for the Turing Test,imitation game.',
  [['',
    'Test',
    'not passive sentence',
    'all the skills needed for the Turing Test'],
   ['', '', 'not passive sentence', 'imitation game.']]],
 ['Eight people, including two children, were injured in the explosion.',
  [['', '', 'not passive sentence', 'Eight people'],
   ['', 'children', 'not passive sentence', ' including two children'],
   ['',
    'explosion',
    'not passive sentence',
    ' were injured in the explosion.']]],
 ['A healthy eating pattern includes fruits, especially whole fruits.',
  [['pattern',
    'fruits',
    'not passive sentence',
    'A healthy eating pattern includes fruits'],
   ['', '', 'not passive sentence', ' especially whole fruits.']]],
 ['mom answers the phone, and the table is broken by marwan.',
  [['mom', 'phone', 'not passive sentence', 'mom answers the phone'],
   ['marwan',
    'table',
    'passive sentence',
    ' and the table is broken by marwan.']]],
 ['today wi

In [ ]:
create_complete_question_function(3)

[['all the skills needed for the Turing Test,imitation game.',
  [['skills needed', 4.0], ['turing test', 4.0], ['imitation game', 4.0]]],
 ['reem is a good person.', [['good person', 4.0], ['reem', 1.0]]],
 ['meshmesh is a good cat.', [['good cat', 4.0], ['meshmesh', 1.0]]],
 ['he eats alot of dry food.', [['eats alot', 4.0], ['dry food', 4.0]]],
 ['GDP in developing countries such as Vietnam will continue growing at a high rate.',
  [['developing countries', 4.0],
   ['continue growing', 4.0],
   ['high rate', 4.0]]],
 ['Here is how you can keep your car and other vehicles clean.',
  [['vehicles clean', 4.0], ['car', 1.0]]],
 ['Here is how you can keep your car or other vehicles clean.',
  [['vehicles clean', 4.0], ['car', 1.0]]],
 ['Eight people, including two children, were injured in the explosion.',
  [['people', 1.0], ['including', 1.0], ['children', 1.0]]],
 ['A healthy eating pattern includes fruits, especially whole fruits.',
  [['healthy eating pattern includes fruits', 23.0